In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
import datetime

In [2]:
data_dir = '/media/real/Samsung_T5/Trash/cropped_faces'
batch_size = 256
epochs = 1000
workers = 0 if os.name == 'nt' else 8

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [4]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir, transform=trans)

In [5]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [6]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

In [7]:
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
    
    # Save the model every 100 epochs
    if (epoch + 1) % 100 == 0:
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        torch.save(resnet.state_dict(), f'finetuned_facenet_weights_epoch{epoch+1}_{current_time}.pth')

writer.close()



Initial
----------
Valid |   471/471  | loss:   10.9677 | fps: 2477.3440 | acc:    0.0000   

Epoch 1/1000
----------
Train |  1884/1884 | loss:    8.2158 | fps:  871.1597 | acc:    0.0489   
Valid |   471/471  | loss:    5.1406 | fps: 2896.9810 | acc:    0.2033   

Epoch 2/1000
----------
Train |  1884/1884 | loss:    2.4412 | fps: 1057.1964 | acc:    0.5642   
Valid |   471/471  | loss:    1.2539 | fps: 2863.3743 | acc:    0.7736   

Epoch 3/1000
----------
Train |  1884/1884 | loss:    0.6405 | fps: 1056.1311 | acc:    0.8703   
Valid |   471/471  | loss:    0.8533 | fps: 2875.6619 | acc:    0.8460   

Epoch 4/1000
----------
Train |  1884/1884 | loss:    0.3759 | fps: 1058.9861 | acc:    0.9208   
Valid |   471/471  | loss:    0.7810 | fps: 2892.6401 | acc:    0.8623   

Epoch 5/1000
----------
Train |  1884/1884 | loss:    0.2770 | fps: 1060.8921 | acc:    0.9392   
Valid |   471/471  | loss:    0.7200 | fps: 2887.8130 | acc:    0.8764   

Epoch 6/1000
----------
Train |  1884/1

Valid |   471/471  | loss:    0.4010 | fps: 2942.6951 | acc:    0.9511   

Epoch 48/1000
----------
Train |  1884/1884 | loss:    0.0028 | fps: 1063.2350 | acc:    0.9994   
Valid |   471/471  | loss:    0.4008 | fps: 2946.1128 | acc:    0.9512   

Epoch 49/1000
----------
Train |  1884/1884 | loss:    0.0028 | fps: 1063.9343 | acc:    0.9994   
Valid |   471/471  | loss:    0.4006 | fps: 2940.0339 | acc:    0.9512   

Epoch 50/1000
----------
Train |  1884/1884 | loss:    0.0027 | fps: 1063.3920 | acc:    0.9994   
Valid |   471/471  | loss:    0.4011 | fps: 2944.5400 | acc:    0.9512   

Epoch 51/1000
----------
Train |  1884/1884 | loss:    0.0026 | fps: 1029.1779 | acc:    0.9994   
Valid |   471/471  | loss:    0.4003 | fps: 2783.9822 | acc:    0.9514   

Epoch 52/1000
----------
Train |  1884/1884 | loss:    0.0026 | fps: 1050.1663 | acc:    0.9994   
Valid |   471/471  | loss:    0.4006 | fps: 2857.5913 | acc:    0.9515   

Epoch 53/1000
----------
Train |  1884/1884 | loss:    

Valid |   471/471  | loss:    0.3919 | fps: 2898.7036 | acc:    0.9545   

Epoch 95/1000
----------
Train |  1884/1884 | loss:    0.0019 | fps: 1062.1196 | acc:    0.9995   
Valid |   471/471  | loss:    0.3927 | fps: 2903.8701 | acc:    0.9545   

Epoch 96/1000
----------
Train |  1884/1884 | loss:    0.0019 | fps: 1062.5145 | acc:    0.9995   
Valid |   471/471  | loss:    0.3919 | fps: 2898.1279 | acc:    0.9545   

Epoch 97/1000
----------
Train |  1884/1884 | loss:    0.0019 | fps: 1060.2501 | acc:    0.9995   
Valid |   471/471  | loss:    0.3917 | fps: 2900.3513 | acc:    0.9546   

Epoch 98/1000
----------
Train |  1884/1884 | loss:    0.0018 | fps: 1061.4128 | acc:    0.9995   
Valid |   471/471  | loss:    0.3926 | fps: 2902.9402 | acc:    0.9545   

Epoch 99/1000
----------
Train |  1884/1884 | loss:    0.0018 | fps: 1062.4956 | acc:    0.9995   
Valid |   471/471  | loss:    0.3928 | fps: 2900.8923 | acc:    0.9548   

Epoch 100/1000
----------
Train |  1884/1884 | loss:   

Train |  1884/1884 | loss:    0.0017 | fps: 1060.6447 | acc:    0.9995   
Valid |   471/471  | loss:    0.3880 | fps: 2886.7642 | acc:    0.9565   

Epoch 142/1000
----------
Train |  1884/1884 | loss:    0.0016 | fps: 1060.0596 | acc:    0.9995   
Valid |   471/471  | loss:    0.3882 | fps: 2886.4780 | acc:    0.9564   

Epoch 143/1000
----------
Train |  1884/1884 | loss:    0.0016 | fps: 1059.9323 | acc:    0.9995   
Valid |   471/471  | loss:    0.3886 | fps: 2887.6858 | acc:    0.9565   

Epoch 144/1000
----------
Train |  1884/1884 | loss:    0.0016 | fps: 1060.0144 | acc:    0.9995   
Valid |   471/471  | loss:    0.3875 | fps: 2884.7700 | acc:    0.9566   

Epoch 145/1000
----------
Train |  1884/1884 | loss:    0.0017 | fps: 1061.1091 | acc:    0.9995   
Valid |   471/471  | loss:    0.3891 | fps: 2884.6531 | acc:    0.9564   

Epoch 146/1000
----------
Train |  1884/1884 | loss:    0.0016 | fps: 1060.7396 | acc:    0.9995   
Valid |   471/471  | loss:    0.3886 | fps: 2887.50

Valid |   471/471  | loss:    0.3860 | fps: 2938.2512 | acc:    0.9574   

Epoch 188/1000
----------
Train |  1884/1884 | loss:    0.0016 | fps: 1064.0570 | acc:    0.9995   
Valid |   471/471  | loss:    0.3868 | fps: 2937.3049 | acc:    0.9575   

Epoch 189/1000
----------
Train |  1884/1884 | loss:    0.0016 | fps: 1064.0664 | acc:    0.9995   
Valid |   471/471  | loss:    0.3867 | fps: 2938.4895 | acc:    0.9575   

Epoch 190/1000
----------
Train |  1884/1884 | loss:    0.0015 | fps: 1063.5256 | acc:    0.9995   
Valid |   471/471  | loss:    0.3871 | fps: 2939.6194 | acc:    0.9574   

Epoch 191/1000
----------
Train |  1884/1884 | loss:    0.0015 | fps: 1063.6843 | acc:    0.9995   
Valid |   471/471  | loss:    0.3875 | fps: 2937.2024 | acc:    0.9575   

Epoch 192/1000
----------
Train |  1884/1884 | loss:    0.0015 | fps: 1063.7448 | acc:    0.9995   
Valid |   471/471  | loss:    0.3864 | fps: 2940.2227 | acc:    0.9576   

Epoch 193/1000
----------
Train |  1884/1884 | los

In [ ]:
print("Finished!!!")